In [1]:
import importlib
from utils import aspect_extraction, sentiment_extraction, general, embeddings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
from gensim.models import KeyedVectors
import gensim
from nltk.corpus import wordnet
import os
import gc
from pathlib import Path

In [2]:
importlib.reload(aspect_extraction)
importlib.reload(sentiment_extraction)
importlib.reload(general)
importlib.reload(embeddings)

<module 'utils.embeddings' from '/home/alessia/Documenti/University/Text mining and sentiment analysis/progetto/tinder/utils/embeddings.py'>

In [3]:
def get_reviews_similarities(app, query, not_query, query_name):
    directory_path = Path(f"data/{app}_embeddings/")
    saving_path = directory_path / f"queries/{query_name}.pickle"

    if os.path.isfile(saving_path):
        return pd.read_pickle(saving_path)
    
    query_similarities = pd.DataFrame() 
    reviews_files = [file for file in directory_path.iterdir() if file.is_file()]

    for i, reviews_file in enumerate(reviews_files):
        print(f"processing reviews {i+1}/{len(reviews_files)}")
        reviews = pd.read_pickle(reviews_file)

        reviews_similarities = embeddings.get_query_similarities(reviews["sentence_embedding"].set_axis(reviews["id"].values), query = query, not_query= not_query, threshold=0.65)
        query_similarities = query_similarities.append(pd.DataFrame(reviews_similarities).reset_index().rename(columns={"index": "id", 0: "similarity"}).merge(reviews, on = "id"))
        gc.collect()
    
    query_similarities = query_similarities.sort_values(by = "similarity", ascending= False)
    if not os.path.isdir(directory_path / "queries"):
        os.mkdir(directory_path / "queries")
    query_similarities.to_pickle(saving_path)
    return query_similarities

## __Tinder__

__Casual sex__

In [ ]:
casual_sex_reviews = get_reviews_similarities(app = "tinder", query= ["casual", "sex"], not_query= ["couple"], query_name= "casual_sex")